## Imputation of 15 remaining item-independent columns

In [67]:
# Load the libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sdv
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    mean_absolute_error, r2_score, root_mean_squared_error
)
from sdv.metadata import SingleTableMetadata
from sdv.single_table import TVAESynthesizer

In [68]:
# Load raw dataset
df_raw = pd.read_csv('dataset_1_item_independent.csv')
df_raw.shape

(3267, 94)

In [69]:
# Columns with missing values
col_miss = df_raw.isna().sum().sort_values()
col_miss.tail(15)

emission_share_agri_waste_mgt              32
total_fdi_inflows                          32
emission_share_farmgate                    32
emission_share_land_use_change             32
emission_share_energy_use                  32
emission_share_crops                       32
emission_share_pre_and_post_production     32
value_added_aff_per_total_fdi              32
emission_share_end_to_end_agrifood         32
emission_share_ipcc_agriculture            32
total_pesticide_export_value               46
phosphorus_production                      47
potassium_agri_use                         48
emission_share_livestock                   54
aoi_credit_to_ag_forest_fish              979
dtype: int64

In [70]:
# Names of the columns that are missing at least one missing value
targets = col_miss.loc[col_miss > 0].keys()
targets

Index(['emission_share_agri_waste_mgt', 'total_fdi_inflows',
       'emission_share_farmgate', 'emission_share_land_use_change',
       'emission_share_energy_use', 'emission_share_crops',
       'emission_share_pre_and_post_production',
       'value_added_aff_per_total_fdi', 'emission_share_end_to_end_agrifood',
       'emission_share_ipcc_agriculture', 'total_pesticide_export_value',
       'phosphorus_production', 'potassium_agri_use',
       'emission_share_livestock', 'aoi_credit_to_ag_forest_fish'],
      dtype='object')

In [117]:
# Work with a copy
df = df_raw.copy()

# Reproducibility
SEED = 42
rng = np.random.default_rng(SEED)

# Pool of rows fully observed on all 15 TARGETS
# --- Excluding the rows for the year 2023 ---
pool_rows = df.loc[(df[targets].notna().all(axis=1)) & (df['year'] != 2023)]
pool_rows.head()

,area,area_code,year_code,year,area_agri_land,area_arable_land,area_cropland,area_with_irrigation,area_permanent_crops,area_temporary_crops,...,least_developed_country,land_locked_developing_country,small_island_developing_state,low_income_food_deficit_country,net_food_importing_developing_country,temp_change_meteorological_year,temp_change_dec_jan_feb,temp_change_jun_jul_aug,temp_change_sep_oct_nov,temp_change_mar_apr_may
29,Albania,3,2007,2007,1119.0,578.0,698.0,356.5,120.0,186.038,...,0.0,0.0,0.0,0.0,0.0,1.389,1.741,2.519,-0.718,2.015
30,Albania,3,2008,2008,1181.0,610.0,697.0,348.0,87.0,206.000,...,0.0,0.0,0.0,0.0,0.0,1.043,0.238,1.862,0.685,1.387
31,Albania,3,2009,2009,1201.3,609.0,696.0,339.5,87.0,202.000,...,0.0,0.0,0.0,0.0,0.0,0.977,0.390,1.261,0.873,1.383
32,Albania,3,2010,2010,1201.3,626.0,696.0,331.0,70.0,202.000,...,0.0,0.0,0.0,0.0,0.0,1.261,1.234,1.607,0.932,1.271
33,Albania,3,2011,2011,1201.0,622.0,696.0,332.0,74.0,205.000,...,0.0,0.0,0.0,0.0,0.0,1.125,0.630,1.659,0.970,1.243


In [72]:
print(f"Total number of rows where all the 15 columns to be imputed are not missing: {pool_rows.shape[0]}") 

Total number of rows where all the 15 columns to be imputed are not missing: 2043


To impute the missing values in the remaining 15 columns, we first pooled the rows 
where all 15 target columns had non-missing values. These complete rows are to be 
used to construct the training and validation datasets. Additionally, to prevent 
data leakage, rows corresponding to the year 2023 were excluded from model training. 

In [118]:
# --- Randomly selecting rows for validation out of pooled rows ---
# For each 'area', pick exactly 1 row at random
val_per_area = (
    pool_rows.groupby('area', group_keys=False).apply(lambda g: g.sample(n=1, random_state=SEED))
)
val_per_area.shape


/var/folders/sh/s_y__9_52fl706bqn_rs0y500000gn/T/ipykernel_37098/2326603602.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  pool_rows.groupby('area', group_keys=False).apply(lambda g: g.sample(n=1, random_state=SEED))


(112, 94)

To construct the validation set, one row corresponding to each country was randomly selected from the pooled set of fully observed rows. This resulted in 112 rows for validation, while the remaining 1,931 rows are to be used for training the imputation 
model.

In [119]:
# Original indices of selected validation rows (these align with the original df)
val_index = val_per_area.index

# carve out validation (clean copy) and training sets
df_val_clean = val_per_area.copy()
df_train = pool_rows.drop(index=val_index).copy()

# Define the categorical columns
categorical_cols = ["area", "region", "sub_region"]

# Move categoricals to string
for c in categorical_cols:
   if c in df_train.columns:
        df_train[c] = df_train[c].astype('category')
   if c in df_val_clean.columns:
        df_val_clean[c] = df_val_clean[c].astype('category')

In [120]:
# Record the mask coordinates (row_id, col) for every TARGET in the val set
mask_records = []
for idx in df_val_clean.index:
    for col in targets:
        mask_records.append((int(idx), col))

# build a table with true values of 15 target cols for scoring later
y_true_df = pd.DataFrame(mask_records, columns=['row_id', 'target'])

In [121]:
# Map each row_id in y_true_df to its integer position in df_val_clean's index
r_idx = df_val_clean.index.get_indexer(y_true_df['row_id'])

# Map each target column name in y_true_df to its integer position in df_val_clean's columns
c_idx = df_val_clean.columns.get_indexer(y_true_df['target'])

# Use NumPy indexing: pull the true values from df_val_clean at the given (row, column) positions
# and store them as a new column 'y_true' in y_true_df
y_true_df['y_true'] = df_val_clean.to_numpy()[r_idx, c_idx]
y_true_df.head()

,row_id,target,y_true
0,29,emission_share_agri_waste_mgt,7.49
1,29,total_fdi_inflows,556.430175
2,29,emission_share_farmgate,53.74
3,29,emission_share_land_use_change,0.0
4,29,emission_share_energy_use,50.75


In [77]:
# Set all target cells in the validation copy to NaN
df_val_masked_lgbm = df_val_clean.copy()
df_val_masked_lgbm.loc[:, targets] = np.nan

In [78]:
# Import libraries for Imputation model and metrics
import lightgbm as lgb
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

In [79]:
# Define the categorical columns
categorical_cols = ["area", "region", "sub_region"]

# Define the columns to be excluded
exclude_cols = ["area_code", "area_code_m49", "year_code"]


# Move categoricals to string
for c in categorical_cols:
   if c in df.columns:
        df[c] = df[c].astype('category')


# Store evaluation metrics for each imputed column
pred_records = []

# Loop over each column that needs imputation
for col in targets:
    # Features: all predictors except the target column and excluded ones
    X_cols = [c for c in df.columns if c != col and c not in exclude_cols]

    # train rows where the target column is observed
    train_mask = df_train[col].notna()
    X_train = df_train.loc[train_mask, X_cols]
    y_train = df_train.loc[train_mask, col]
    
    # validation rows where the target is not observed
    X_val = df_val_masked_lgbm.loc[:, X_cols]

    # Choose objective
    y_min = y_train.min()

    if y_min >= 0:
        obj = "tweedie"
    else:
        obj = "regression"

    model = lgb.LGBMRegressor(
        objective=obj,
        n_estimators=800,
        random_state=SEED,
        n_jobs=-1,
        verbosity=-1
    )

    model.fit(X_train, y_train, categorical_feature=categorical_cols)

    # predict for validation set rows
    y_pred = np.round(model.predict(X_val), 3)

    # Collect long-form predictions
    pred_records.extend([
        {"row_id": int(rid), "target": col, "method": "LightGBM", "y_pred": float(p)}
        for rid, p in zip(X_val.index, y_pred)
    ])

# Assemble predictions for downstream metrics merge
y_pred_df_lgbm = pd.DataFrame(pred_records, columns=["row_id", "target", "method", "y_pred"])
y_pred_df_lgbm.head(10)

,row_id,target,method,y_pred
0,29,emission_share_agri_waste_mgt,LightGBM,7.271
1,57,emission_share_agri_waste_mgt,LightGBM,0.906
2,60,emission_share_agri_waste_mgt,LightGBM,6.962
3,103,emission_share_agri_waste_mgt,LightGBM,3.811
4,106,emission_share_agri_waste_mgt,LightGBM,7.618
5,129,emission_share_agri_waste_mgt,LightGBM,-0.363
6,179,emission_share_agri_waste_mgt,LightGBM,2.690
7,198,emission_share_agri_waste_mgt,LightGBM,8.859
8,221,emission_share_agri_waste_mgt,LightGBM,7.742
9,244,emission_share_agri_waste_mgt,LightGBM,-0.030


In [80]:
# Merge truth and predictions (only LightGBM here)
eval_df = (
    y_true_df.merge(
        y_pred_df_lgbm[["row_id", "target", "y_pred"]],
        on=["row_id", "target"],
        how="inner"
    )
)
eval_df.head(10)

,row_id,target,y_true,y_pred
0,29,emission_share_agri_waste_mgt,7.49,7.271
1,29,total_fdi_inflows,556.430175,-9009.922
2,29,emission_share_farmgate,53.74,30.731
3,29,emission_share_land_use_change,0.0,0.424
4,29,emission_share_energy_use,50.75,33.629
5,29,emission_share_crops,2.83,1.990
6,29,emission_share_pre_and_post_production,15.7,13.083
7,29,value_added_aff_per_total_fdi,3.003491,502.700
8,29,emission_share_end_to_end_agrifood,69.44,4.578
9,29,emission_share_ipcc_agriculture,47.88,-1.208


In [81]:
# per-target metrics table
metrics_rows = []
for col in targets:
    sub = eval_df.loc[eval_df["target"]==col]
    
    y_val = sub["y_true"].astype(float)
    val_pred = sub["y_pred"].astype(float)

    # Metrics
    rmse = root_mean_squared_error(y_val, val_pred)
    mae = mean_absolute_error(y_val, val_pred)
    r2  = r2_score(y_val, val_pred)

    # Normalizations
    mean_y = float(y_val.mean())
    std_y  = float(y_val.std(ddof=1))  # sample std (ddof=1) is typical; either is fine if consistent

    nrmse_mean = (rmse / mean_y) if mean_y != 0 else np.nan
    nrmse_std  = (rmse / std_y)  if std_y  != 0 else np.nan

    # Mean Absolute Percentage Error (ignoring inf/NaN cases)
    mape = (np.abs((y_val - val_pred)/y_val)
            .replace([np.inf, -np.inf], np.nan)
            .dropna()
            .mean()*100)

    # n_train for this target = observed count in TRAIN
    n_train = int(df_train[col].notna().sum())
    n_val   = int(len(y_val))  

    metrics_rows.append({
        "method": "LightGBM",
        "target": col,
        "n_train": n_train,
        "n_val": n_val,
        "RMSE": np.round(rmse, 3),
        "MAE": np.round(mae, 3),
        "R2": np.round(r2, 3),
        "nRMSE_mean": np.round(nrmse_mean, 3),
        "nRMSE_std": np.round(nrmse_std, 3),
        "MAPE(%)": np.round(mape, 3)
    })

metrics_lgbm = pd.DataFrame(metrics_rows, columns=[
    "method","target","n_train","n_val","RMSE","MAE","R2","nRMSE_mean","nRMSE_std","MAPE(%)"
])
metrics_lgbm

,method,target,n_train,n_val,RMSE,MAE,R2,nRMSE_mean,nRMSE_std,MAPE(%)
0,LightGBM,emission_share_agri_waste_mgt,1931,112,4.289,3.363,0.327,0.542,0.817,59.518
1,LightGBM,total_fdi_inflows,1931,112,27954.532,15992.946,-0.123,3.419,1.055,8325.723
2,LightGBM,emission_share_farmgate,1931,112,23.018,18.722,-0.221,0.891,1.100,136.546
3,LightGBM,emission_share_land_use_change,1931,112,8.418,6.517,0.789,0.742,0.457,2848.870
4,LightGBM,emission_share_energy_use,1931,112,13.905,11.336,0.770,0.250,0.477,39.314
5,LightGBM,emission_share_crops,1931,112,0.741,0.524,0.716,0.490,0.530,136.706
6,LightGBM,emission_share_pre_and_post_production,1931,112,4.884,4.142,0.577,0.402,0.648,45.668
7,LightGBM,value_added_aff_per_total_fdi,1931,112,1296.910,918.799,-268.365,64.465,16.339,223479.199
8,LightGBM,emission_share_end_to_end_agrifood,1931,112,33.087,26.820,-0.463,0.671,1.204,69.466
9,LightGBM,emission_share_ipcc_agriculture,1931,112,26.769,20.610,-0.967,1.301,1.396,226.733


## Tabular Variational Auto-Encoder

To impute the missing values in the remaining 15 columns, we experimented with 
the Tabular Variational Autoencoder (TVAE)- a deep generative model that, unlike 
LightGBM or KNN, does not predict missing values directly. Instead, TVAE learns 
the joint probability distribution of all features in the dataset and can then 
sample new rows from this learned distribution to fill in missing values in a 
way that is statistically consistent with the rest of the data.

It is important to note that when sampling to impute missing values, we must 
provide a conditional context, that is, we specify the values of certain columns 
and ask the model to infer what the remaining values should be, given those 
conditions. If we simply input an entire row containing missing entries as the 
conditional, the model may fail to converge because it has not encountered that 
specific pattern of column values during training. Therefore, the choice of 
conditional columns plays a crucial role in whether the model can successfully 
generate plausible imputations. Additionally, since TVAE samples entire rows 
rather than individual values, we must ensure that only the columns being imputed 
are replaced with the newly generated values. 


In [82]:
# Import libraries
from sdv.metadata import SingleTableMetadata
from sdv.single_table import TVAESynthesizer

In [83]:
# Set the seed for replication
rng = np.random.default_rng(SEED)

In [84]:
# Prepare train data
tv_train = df_train.drop(columns=[c for c in exclude_cols if c in df_train.columns], errors="ignore").copy()

tv_train = df_train.loc[df_train['aoi_credit_to_ag_forest_fish'].notna()]

# Year to numeric
tv_train["year"] = pd.to_numeric(tv_train["year"], errors="coerce")

# Ensure SDV-friendly dtypes for declared categoricals
for c in categorical_cols:
    if c in tv_train.columns:
        tv_train[c] = tv_train[c].astype("object")

# identify numeric columns
numeric_cols = tv_train.select_dtypes(include=[np.number]).columns.to_list()

# detect binary {0,1} among numeric columns
binary_cols = []
for c in numeric_cols:
    uv = pd.Series(tv_train[c]).dropna().unique()
    if len(uv) > 0 and set(uv).issubset({0, 1, 0.0, 1.0}):
        binary_cols.append(c)
        tv_train[c] = tv_train[c].astype("Int64").astype("boolean")

# Build and lock Metadata
md = SingleTableMetadata()
md.detect_from_dataframe(tv_train)  # baseline detection

# Force sdtypes per our simple rules
for c in tv_train.columns:
    if c in categorical_cols:
        md.update_column(c, sdtype="categorical")
    elif c in binary_cols:
        md.update_column(c, sdtype="boolean")
    else:
        md.update_column(c, sdtype="numerical")

md.validate()


In [ ]:
# Fit TVAE on train
np.random.seed(SEED)
synth = TVAESynthesizer(
    metadata=md,
    epochs=100,            
    batch_size=512,
    embedding_dim=64,
    compress_dims=(128, 64),
    decompress_dims=(64, 128),
    l2scale=1e-5,
    verbose=False,
    cuda=None   
)

synth.fit(tv_train)

In [86]:
# Set all target cells in the validation copy to NaN
df_val_masked = df_val_clean.copy()
df_val_masked.loc[:, targets] = np.nan

# Step 3a: Build light conditions with only 1 categoricals fixed
categorical_keys = ["area"]

# Make a conditions copy the same shape as df
tv_val_masked = df_val_masked.drop(columns=[c for c in exclude_cols if c in df_val_masked.columns], errors="ignore").copy()

# Coerce 'year' numeric & categoricals to object to mirror TRAIN
if "year" in tv_val_masked.columns:
    tv_val_masked["year"] = pd.to_numeric(tv_val_masked["year"], errors="coerce")
for c in categorical_cols:
    if c in tv_val_masked.columns:
        tv_val_masked[c] = tv_val_masked[c].astype("object")


conditions_df = tv_val_masked.copy()

# 1) Work only on rows that have ANY missing among the targets
need_mask = conditions_df[targets].isna().any(axis=1)
need_idx = conditions_df.index[need_mask]

# 2) Build known-conditions for *those* rows (keys have no missing)
known = conditions_df.loc[need_idx, categorical_keys].copy()

# 3) Sample remaining columns for exactly those rows
samples = synth.sample_remaining_columns(
    known_columns=known,
    batch_size=512,          
    max_tries_per_batch=300
)


Sampling remaining columns:  71%|███████▏  | 80/112 [1:15:57<30:22, 56.96s/it]  
/Users/gurjitsingh/opt/anaconda3/envs/sdv_impute/lib/python3.12/site-packages/sdv/single_table/utils.py:158: UserWarning: Only able to sample 80 rows for the given conditions. To sample more rows, try increasing `max_tries_per_batch` (currently: 300). Note that increasing this value will also increase the sampling time.
  warnings.warn(user_msg)


When we fitted the TVAE model using 'area' as the conditional column, it successfully generated 85 out of 112 rows in the validation set. For the remaining rows, it is 
likely that the model had not encountered enough examples in the training data corresponding to those specific 'area' values. To recover these missing rows, we 
next use 'sub_region' as the conditional variable. Since 'sub_region' has considerably fewer levels than 'area', we expect that the model has seen enough examples for each sub-region level in the training set to generate new, valid samples.  

In [87]:
samples.index = samples.index.astype('int')
samples.head()

,area,area_code,year_code,year,area_agri_land,area_arable_land,area_cropland,area_with_irrigation,area_permanent_crops,area_temporary_crops,...,least_developed_country,land_locked_developing_country,small_island_developing_state,low_income_food_deficit_country,net_food_importing_developing_country,temp_change_meteorological_year,temp_change_dec_jan_feb,temp_change_jun_jul_aug,temp_change_sep_oct_nov,temp_change_mar_apr_may
57,Angola,108,2011,2011,48805.544791,6608.9967,9254.3076,381.442228,0.1000,737.1886,...,False,False,False,False,False,1.215597,0.460730,1.096567,2.394922,1.782770
60,Antigua and Barbuda,26,2009,2018,0.660000,0.1350,0.6600,61.557315,162.8570,0.1330,...,False,False,True,False,True,0.973904,0.622002,0.384870,0.590602,0.103042
103,Argentina,14,2012,2007,6690.276121,1170.2492,2827.1727,370.926912,161.7552,976.7501,...,False,False,False,False,False,0.959786,0.338089,1.154109,0.890131,0.491640
106,Armenia,82,2022,2022,4660.149771,930.6943,1144.3607,444.568797,83.6336,665.0259,...,False,False,False,False,True,1.076590,1.442054,0.900711,0.588114,1.150365
129,Australia,24,2005,2014,40772.160294,39242.1091,17346.4206,877.008297,984.2090,34752.9681,...,False,False,False,False,False,0.788082,0.653085,0.808102,0.909187,0.349535


In [ ]:
# Get the index of the rows that are imputed
samples_idx = samples.index
samples_idx

80

In [89]:
# Fill the missing values with predictions
for col in targets:
    na_mask = conditions_df.loc[samples_idx, col].isna()
    conditions_df.loc[samples_idx[na_mask], col] = samples.loc[samples_idx[na_mask], col].values

# Calculate the remaining number of missing values
conditions_df[targets].isna().any(axis=1).sum()

32

In [91]:
# Work with a copy()
df_val_masked = conditions_df.copy()

In [92]:
"""
Fill the remaining missing values with a light conditional- 'sub_region' instead 
of 'area'
"""

# Build light conditions with only 1 categoricals fixed
categorical_keys = ["sub_region"]

# Make a conditions copy the same shape as df
tv_val_masked = df_val_masked.drop(columns=[c for c in exclude_cols if c in df_val_masked.columns], errors="ignore").copy()

# Coerce 'year' numeric & categoricals to object to mirror TRAIN
if "year" in tv_val_masked.columns:
    tv_val_masked["year"] = pd.to_numeric(tv_val_masked["year"], errors="coerce")
for c in categorical_cols:
    if c in tv_val_masked.columns:
        tv_val_masked[c] = tv_val_masked[c].astype("object")


conditions_df = tv_val_masked.copy()

# 1) Work only on rows that have ANY missing among the targets
need_mask = conditions_df[targets].isna().any(axis=1)
need_idx = conditions_df.index[need_mask]

# 2) Build known-conditions for *those* rows (keys have no missing)
known = conditions_df.loc[need_idx, categorical_keys].copy()

# 3) Sample remaining columns for exactly those rows
samples_2 = synth.sample_remaining_columns(
    known_columns=known,
    batch_size=512,          
    max_tries_per_batch=300
)

Sampling remaining columns:  97%|█████████▋| 31/32 [02:52<00:05,  5.57s/it]
/Users/gurjitsingh/opt/anaconda3/envs/sdv_impute/lib/python3.12/site-packages/sdv/single_table/utils.py:158: UserWarning: Only able to sample 31 rows for the given conditions. To sample more rows, try increasing `max_tries_per_batch` (currently: 300). Note that increasing this value will also increase the sampling time.
  warnings.warn(user_msg)


In [93]:
samples_2_idx = samples_2.index
samples_2_idx

Index([  29.0,  221.0,  393.0,  444.0,  483.0,  536.0,  557.0,  569.0, 1063.0,
       1082.0, 1089.0, 1135.0, 1216.0, 1284.0, 1976.0, 2027.0, 2088.0, 2125.0,
       2146.0, 2251.0, 2328.0, 2413.0, 2451.0, 2551.0, 2591.0, 2661.0, 2892.0,
       2965.0, 2995.0, 3087.0, 3218.0],
      dtype='float64')

In [94]:
# Fill the missing values with predictions
for col in targets:
    na_mask = conditions_df.loc[samples_2_idx, col].isna()
    conditions_df.loc[samples_2_idx, col] = samples_2.loc[samples_2_idx, col].values

# Calculate the remaining number of missing values
conditions_df[targets].isna().any(axis=1).sum()

1

In [95]:
# Work with a copy()
df_val_masked = conditions_df.copy()

In [96]:
"""
Fill the last two remaining rows with an even lighter conditional- 
'region' instead of 'sub_region' and 'area'
"""

# Step 3a: Build light conditions with only 1 categoricals fixed
categorical_keys = ["region"]

# Make a conditions copy the same shape as df
tv_val_masked = df_val_masked.drop(columns=[c for c in exclude_cols if c in df_val_masked.columns], errors="ignore").copy()

# Coerce 'year' numeric & categoricals to object to mirror TRAIN
if "year" in tv_val_masked.columns:
    tv_val_masked["year"] = pd.to_numeric(tv_val_masked["year"], errors="coerce")
for c in categorical_cols:
    if c in tv_val_masked.columns:
        tv_val_masked[c] = tv_val_masked[c].astype("object")


conditions_df = tv_val_masked.copy()

# 1) Work only on rows that have ANY missing among the targets
need_mask = conditions_df[targets].isna().any(axis=1)
need_idx = conditions_df.index[need_mask]

# 2) Build known-conditions for *those* rows (keys have no missing)
known = conditions_df.loc[need_idx, categorical_keys].copy()

# 3) Sample remaining columns for exactly those rows
samples_3 = synth.sample_remaining_columns(
    known_columns=known,
    batch_size=512,          
    max_tries_per_batch=300
)

Sampling remaining columns: 100%|██████████| 1/1 [01:00<00:00, 60.74s/it]


After using 'area' and then 'sub_region' as conditional columns, the TVAE model 
was able to generate 110 out of 112 rows in the validation set. The remaining 
two rows were produced by switching the conditional to 'region', which has even 
fewer levels than 'sub_region'. Following this stepwise approach, we successfully 
generated all rows in the validation set.

In [97]:
samples_3_idx = samples_3.index

# Fill the missing values with predictions
for col in targets:
    na_mask = conditions_df.loc[samples_3_idx, col].isna()
    conditions_df.loc[samples_3_idx, col] = samples_3.loc[samples_3_idx, col].values

df_val_masked = conditions_df.copy()

In [122]:
y_true_df.head()

,row_id,target,y_true
0,29,emission_share_agri_waste_mgt,7.49
1,29,total_fdi_inflows,556.430175
2,29,emission_share_farmgate,53.74
3,29,emission_share_land_use_change,0.0
4,29,emission_share_energy_use,50.75


In [99]:
# Map each row_id in y_true_df to its integer position in df_val_clean's index
r_idx = df_val_masked.index.get_indexer(y_true_df['row_id'])

# Map each target column name in y_true_df to its integer position in df_val_clean's columns
c_idx = df_val_masked.columns.get_indexer(y_true_df['target'])

# Use NumPy indexing: pull the true values from df_val_clean at the given (row, column) positions
# and store them as a new column 'y_true' in y_true_df
y_true_df['y_pred'] = df_val_masked.to_numpy()[r_idx, c_idx]
eval_df = y_true_df.copy()
eval_df.head(10)

,row_id,target,y_true,y_pred
0,29,emission_share_agri_waste_mgt,7.49,1.15
1,29,total_fdi_inflows,556.430175,23960.058655
2,29,emission_share_farmgate,53.74,17.56
3,29,emission_share_land_use_change,0.0,-0.42
4,29,emission_share_energy_use,50.75,80.26
5,29,emission_share_crops,2.83,1.38
6,29,emission_share_pre_and_post_production,15.7,10.78
7,29,value_added_aff_per_total_fdi,3.003491,10.391634
8,29,emission_share_end_to_end_agrifood,69.44,27.4
9,29,emission_share_ipcc_agriculture,47.88,5.44


In [100]:
# per-target metrics table
metrics_rows = []
for col in targets:
    sub = eval_df.loc[eval_df["target"]==col]
    
    y_val = sub["y_true"].astype(float)
    val_pred = sub["y_pred"].astype(float)

    # Metrics
    rmse = root_mean_squared_error(y_val, val_pred)
    mae = mean_absolute_error(y_val, val_pred)
    r2  = r2_score(y_val, val_pred)

    # Normalizations
    mean_y = float(y_val.mean())
    std_y  = float(y_val.std(ddof=1))  # sample std (ddof=1) is typical; either is fine if consistent

    nrmse_mean = (rmse / mean_y) if mean_y != 0 else np.nan
    nrmse_std  = (rmse / std_y)  if std_y  != 0 else np.nan

    # Mean Absolute Percentage Error (ignoring inf/NaN cases)
    mape = (np.abs((y_val - val_pred)/y_val)
            .replace([np.inf, -np.inf], np.nan)
            .dropna()
            .mean()*100)

    # n_train for this target = observed count in TRAIN
    n_train = int(df_train[col].notna().sum())
    n_val   = int(len(y_val))  

    metrics_rows.append({
        "method": "TVAE",
        "target": col,
        "n_train": n_train,
        "n_val": n_val,
        "RMSE": np.round(rmse, 3),
        "MAE": np.round(mae, 3),
        "R2": np.round(r2, 3),
        "nRMSE_mean": np.round(nrmse_mean, 3),
        "nRMSE_std": np.round(nrmse_std, 3),
        "MAPE(%)": np.round(mape, 3)
    })

metrics_tvae = pd.DataFrame(metrics_rows, columns=[
    "method","target","n_train","n_val","RMSE","MAE","R2","nRMSE_mean","nRMSE_std","MAPE(%)"
])
metrics_tvae

,method,target,n_train,n_val,RMSE,MAE,R2,nRMSE_mean,nRMSE_std,MAPE(%)
0,TVAE,emission_share_agri_waste_mgt,1931,112,6.329,4.480,-0.466,0.800,1.206,69.661
1,TVAE,total_fdi_inflows,1931,112,27401.557,10396.864,-0.079,3.352,1.034,2198.009
2,TVAE,emission_share_farmgate,1931,112,20.895,14.439,-0.006,0.809,0.998,320.482
3,TVAE,emission_share_land_use_change,1931,112,19.879,10.498,-0.177,1.752,1.080,263.659
4,TVAE,emission_share_energy_use,1931,112,25.739,17.092,0.212,0.463,0.884,67.832
5,TVAE,emission_share_crops,1931,112,1.457,0.977,-0.098,0.964,1.043,362.985
6,TVAE,emission_share_pre_and_post_production,1931,112,8.052,5.936,-0.150,0.663,1.068,58.640
7,TVAE,value_added_aff_per_total_fdi,1931,112,80.332,22.682,-0.033,3.993,1.012,2981.909
8,TVAE,emission_share_end_to_end_agrifood,1931,112,25.893,18.833,0.104,0.525,0.942,55.447
9,TVAE,emission_share_ipcc_agriculture,1931,112,19.178,12.503,-0.010,0.932,1.000,548.143


In [101]:
# Save results of TVAE imputer as an html file
metrics_tvae.to_html('metrics_tvae.html', index=False)

In [53]:
# Save results of LGBM imputer as an html file
metrics_lgbm.to_html('metrics_lgbm.html', index=False)

## K-NN Imputer

Another model we used to impute missing values in the remaining 15 columns was the K-Nearest Neighbors (KNN) Imputer, a non-parametric method that fills in missing 
entries by referencing the rows most similar to the one with the missing value. 
Similarity between rows is typically measured using Euclidean distance, calculated 
over the non-missing features. When a row contains multiple missing cells, each 
cell is imputed independently, meaning that different cells may be filled using 
different sets of nearest neighbors.

In [110]:
# Load required libraries
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [111]:
# Load raw dataset
df_raw = pd.read_csv('dataset_1_item_independent.csv')

# define the target columns which have at least one missing value
col_miss = df_raw.isna().sum().sort_values() 
targets= col_miss.loc[col_miss > 0].keys() 

# work with a copy 
df = df_raw.copy()  

# Pool of rows fully observed on all 15 TARGETS and skipping the year 2023 to avoid
# data leakage
pool_rows = df.loc[(df[targets].notna().all(axis=1)) & (df['year'] != 2023)]

# For each 'area', pick exactly 1 row at random 
val_per_area = (pool_rows.groupby('area', group_keys=False).apply(lambda g: g.sample(n=1, random_state=SEED)) ) 

# Define the categorical variables explicitly
categorical_cols = ["area", "region", "sub_region"] 
for c in categorical_cols: 
    if c in df.columns: 
        df[c] = df[c].astype('category') 
        
# Original indices of selected validation rows (these align with the original df) 
val_index = val_per_area.index 

# carve out validation (clean copy) and training sets 
df_val_clean = val_per_area.copy() 
df_train = pool_rows.drop(index=val_index).copy()

/var/folders/sh/s_y__9_52fl706bqn_rs0y500000gn/T/ipykernel_37098/825198081.py:16: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  val_per_area = (pool_rows.groupby('area', group_keys=False).apply(lambda g: g.sample(n=1, random_state=SEED)) )


In [112]:
# Columns to be excluded
exclude_cols = ["area_code", "area_code_m49", "year_code"]

# Categorical columns
categorical_cols = ["area", "region", "sub_region"]

# Columns used to build the KNN distance space
feature_cols = [c for c in df.columns if c not in exclude_cols]

# Identify dtypes off your working df 
num_cols = df.select_dtypes('number').columns.to_list()
num_cols = [c for c in num_cols if c not in exclude_cols and c not in categorical_cols]

cat_cols = [c for c in categorical_cols if c in feature_cols]

# encode categorical columns and scale numericals
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
        ("num", StandardScaler(), num_cols),
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

# --- Donor pool: fully observed across the entire distance space ---
donor_mask = df_train[feature_cols].notna().all(axis=1)
df_donors = df_train.loc[donor_mask, feature_cols].copy()

# Validation copy (for KNN): mask the same target cols as before
df_val_masked_knn = df_val_clean.copy()
df_val_masked_knn.loc[:, targets] = np.nan

print("Donors:", df_donors.shape, " | Train (post-split):", df_train.shape)

Donors: (1931, 91)  | Train (post-split): (1931, 94)


In [105]:
# Import KNNImputer
from sklearn.impute import KNNImputer

In [113]:
# Fit the pre-processor on donors
preprocessor.fit(df_donors)

# Transform donors and validation
X_donors = preprocessor.transform(df_donors)
X_val = preprocessor.transform(df_val_masked_knn[feature_cols])

# Fit KNNImputer on donor space, then impute the validation rows
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputer.fit(X_donors)

X_val_imp = imputer.transform(X_val)

In [114]:
# Get the names of features after preprocessing
feature_names = preprocessor.get_feature_names_out()

# Identify which positions correspond to our numeric cols
num_positions = [i for i, f in enumerate(feature_names) if f in num_cols]

X_val_num_imputed = X_val_imp[:, num_positions]

# 3) Inverse-scale the numeric slice back to original units
scaler = preprocessor.named_transformers_["num"]  # StandardScaler
X_val_num_imputed = scaler.inverse_transform(X_val_num_imputed)

# Build back a dataframe of imputed numerics
df_val_imputed_num = pd.DataFrame(
    X_val_num_imputed,
    index = df_val_masked_knn.index,
    columns = num_cols
)

# From these, extract only the target cols
df_val_imputed_targets = df_val_imputed_num[targets].copy()

In [115]:
# Collect predictions for just target columns
pred_records = []
for idx in df_val_imputed_targets.index:
    for col in targets:
        pred_records.append({
            "row_id": int(idx),
            "target": col,
            "method": "KNN",
            "y_pred": float(df_val_imputed_targets.loc[idx, col])
        })

y_pred_df_knn = pd.DataFrame(pred_records, columns=["row_id", "target", "method", "y_pred"])
y_pred_df_knn.head(10)

,row_id,target,method,y_pred
0,29,emission_share_agri_waste_mgt,KNN,7.298000
1,29,total_fdi_inflows,KNN,844.244812
2,29,emission_share_farmgate,KNN,45.508000
3,29,emission_share_land_use_change,KNN,0.000000
4,29,emission_share_energy_use,KNN,52.054000
5,29,emission_share_crops,KNN,2.916000
6,29,emission_share_pre_and_post_production,KNN,15.800000
7,29,value_added_aff_per_total_fdi,KNN,2.376961
8,29,emission_share_end_to_end_agrifood,KNN,61.308000
9,29,emission_share_ipcc_agriculture,KNN,40.050000


In [123]:
y_true_df.head()

,row_id,target,y_true
0,29,emission_share_agri_waste_mgt,7.49
1,29,total_fdi_inflows,556.430175
2,29,emission_share_farmgate,53.74
3,29,emission_share_land_use_change,0.0
4,29,emission_share_energy_use,50.75


In [124]:
# Build metrics_knn

# Merge truth and KNN predictions
eval_df_knn = (
    y_true_df.merge(
        y_pred_df_knn[['row_id', 'target', 'y_pred']],
        on=['row_id', 'target'],
        how='inner'
    )
)
eval_df_knn.head(10)

,row_id,target,y_true,y_pred
0,29,emission_share_agri_waste_mgt,7.49,7.298000
1,29,total_fdi_inflows,556.430175,844.244812
2,29,emission_share_farmgate,53.74,45.508000
3,29,emission_share_land_use_change,0.0,0.000000
4,29,emission_share_energy_use,50.75,52.054000
5,29,emission_share_crops,2.83,2.916000
6,29,emission_share_pre_and_post_production,15.7,15.800000
7,29,value_added_aff_per_total_fdi,3.003491,2.376961
8,29,emission_share_end_to_end_agrifood,69.44,61.308000
9,29,emission_share_ipcc_agriculture,47.88,40.050000


In [125]:
# Per-target metrics loop (mirrors your LightGBM block)
metrics_rows = []
for col in targets:
    sub = eval_df_knn.loc[eval_df_knn["target"] == col]

    y_val   = sub["y_true"].astype(float)
    val_pred = sub["y_pred"].astype(float)

    # Metrics
    rmse = root_mean_squared_error(y_val, val_pred)
    mae  = mean_absolute_error(y_val, val_pred)
    r2   = r2_score(y_val, val_pred)

    # Normalizations
    mean_y = float(y_val.mean())
    std_y  = float(y_val.std(ddof=1))

    nrmse_mean = (rmse / mean_y) if mean_y != 0 else np.nan
    nrmse_std  = (rmse / std_y)  if std_y  != 0 else np.nan

    mape = (np.abs((y_val - val_pred)/y_val)
            .replace([np.inf, -np.inf], np.nan)
            .dropna()
            .mean()*100)

    n_train = int(df_train[col].notna().sum())
    n_val   = int(len(y_val))

    metrics_rows.append({
        "method": "KNN",
        "target": col,
        "n_train": n_train,
        "n_val": n_val,
        "RMSE": np.round(rmse, 3),
        "MAE": np.round(mae, 3),
        "R2": np.round(r2, 3),
        "nRMSE_mean": np.round(nrmse_mean, 3),
        "nRMSE_std": np.round(nrmse_std, 3),
        "MAPE(%)": np.round(mape, 3)
    })

metrics_knn = pd.DataFrame(metrics_rows, columns=[
    "method","target","n_train","n_val","RMSE","MAE","R2","nRMSE_mean","nRMSE_std","MAPE(%)"
])
metrics_knn

,method,target,n_train,n_val,RMSE,MAE,R2,nRMSE_mean,nRMSE_std,MAPE(%)
0,KNN,emission_share_agri_waste_mgt,1931,112,2.597,1.321,0.753,0.328,0.495,21.292
1,KNN,total_fdi_inflows,1931,112,14951.699,4422.224,0.679,1.829,0.564,208.496
2,KNN,emission_share_farmgate,1931,112,9.123,3.989,0.808,0.353,0.436,36.637
3,KNN,emission_share_land_use_change,1931,112,4.364,2.007,0.943,0.385,0.237,164.649
4,KNN,emission_share_energy_use,1931,112,9.111,4.924,0.901,0.164,0.313,12.799
5,KNN,emission_share_crops,1931,112,0.486,0.286,0.878,0.322,0.348,87.037
6,KNN,emission_share_pre_and_post_production,1931,112,2.591,1.464,0.881,0.213,0.344,16.713
7,KNN,value_added_aff_per_total_fdi,1931,112,361.075,51.790,-19.879,17.948,4.549,327.505
8,KNN,emission_share_end_to_end_agrifood,1931,112,9.125,4.886,0.889,0.185,0.332,17.771
9,KNN,emission_share_ipcc_agriculture,1931,112,8.223,2.958,0.814,0.400,0.429,43.291


In [126]:
# Save results of KNN Imputer as an html file
metrics_knn.to_html('metrics_knn.html', index=False)

## Final Imputation of item-independent columns

Upon carefully examining the validation results from the three imputation 
techniques- 'LightGBM', 'Tabular Variational Auto Encoder (TVAE)', and 'KNN-Imputer', 
we can clearly notice that the KNN Imputer outperforms for almost all the columns, 
with the only exception of the column- 'potassium_agri_use'. 

The decisions are made primarily based on the metrics- 'RMSE', 'MAE', 'R2', and 
'nRMSE_std'. 'MAPE' is avoided because it is unreliable in our case as many of 
the columns have either zeros or near-zero values, which explodes the percentage error.

So, the key takeaways are- 
- KNN outperforms on all emission-share columns and most continuous targets.
- LightGBM only for potassium_agri_use
- TVAE is generally not competitive for direct point imputation here

In [448]:
# List of columns to be imputed with LightGBM model
lgbm_cols = [
'potassium_agri_use'
]

# List of columns to be imputed with KNN model
knn_cols = [
    'aoi_credit_to_ag_forest_fish', 'emission_share_land_use_change', 
    'total_fdi_inflows', 'total_pesticide_export_value', 'phosphorus_production',
    'emission_share_agri_waste_mgt', 'emission_share_farmgate', 
    'emission_share_energy_use', 'emission_share_crops', 
    'emission_share_pre_and_post_production', 'emission_share_end_to_end_agrifood',
    'emission_share_ipcc_agriculture', 'emission_share_livestock'
    ]

In [449]:
# Load raw dataset
df_raw = pd.read_csv('dataset_1_item_independent.csv')

df_raw = df_raw.drop('value_added_aff_per_total_fdi', axis=1)
df_raw.head()

,area,area_code,year_code,year,area_agri_land,area_arable_land,area_cropland,area_with_irrigation,area_permanent_crops,area_temporary_crops,...,least_developed_country,land_locked_developing_country,small_island_developing_state,low_income_food_deficit_country,net_food_importing_developing_country,temp_change_meteorological_year,temp_change_dec_jan_feb,temp_change_jun_jul_aug,temp_change_sep_oct_nov,temp_change_mar_apr_may
0,Afghanistan,2,2001,2001,37795.0,7683.0,7795.0,3203.0,112.0,2502.0,...,1.0,1.0,0.0,1.0,1.0,1.377,0.433,1.090,1.209,2.778
1,Afghanistan,2,2002,2002,37790.0,7678.0,7790.0,3208.0,112.0,2111.0,...,1.0,1.0,0.0,1.0,1.0,1.457,1.722,0.991,1.589,1.524
2,Afghanistan,2,2003,2003,37884.0,7772.0,7884.0,3208.0,112.0,3761.0,...,1.0,1.0,0.0,1.0,1.0,0.710,1.105,0.947,0.787,0.002
3,Afghanistan,2,2004,2004,37928.0,7816.0,7928.0,3208.0,112.0,2934.0,...,1.0,1.0,0.0,1.0,1.0,1.482,1.959,0.789,1.182,1.999
4,Afghanistan,2,2005,2005,37917.0,7805.0,7917.0,3208.0,112.0,3385.0,...,1.0,1.0,0.0,1.0,1.0,0.513,-0.305,0.702,1.306,0.348


In [ ]:
# Columns with percentage of missingness
(df_raw.isna().mean()*100).sort_values(ascending=False).head(15)

aoi_credit_to_ag_forest_fish              29.966330
emission_share_livestock                   1.652893
potassium_agri_use                         1.469238
phosphorus_production                      1.438629
total_pesticide_export_value               1.408020
emission_share_agri_waste_mgt              0.979492
emission_share_land_use_change             0.979492
emission_share_pre_and_post_production     0.979492
emission_share_end_to_end_agrifood         0.979492
emission_share_crops                       0.979492
total_fdi_inflows                          0.979492
emission_share_ipcc_agriculture            0.979492
emission_share_energy_use                  0.979492
emission_share_farmgate                    0.979492
cropland_nitrogen_use_efficiency           0.000000
dtype: float64

In [450]:
df = df_raw.copy()

categorical_cols = ["area", "region", "sub_region"]
exclude_cols = ["area_code", "area_code_m49", "year_code"]

for c in categorical_cols:
    if c in df.columns:
        df[c] = df[c].astype('category')

# Loop over each column that needs to be imputed using LightGBM
for col in lgbm_cols:
    # Features: all predictors except the target column and excluded ones
    X_cols = [c for c in df.columns if c != col and c not in exclude_cols]

    # train rows where the target column is observed
    train_mask = df[col].notna()
    X_train = df.loc[train_mask, X_cols]
    y_train = df.loc[train_mask, col]

    # test rows where the target is not observed/missing
    test_mask = df[col].isna()
    X_test = df.loc[test_mask, X_cols]

    # Choose objective
    if y_train.min() >= 0:
        obj = "tweedie"
    else:
        obj = "regression"

    # Tell LGBM which columns are categorical and present in X
    cat_in_X = [c for c in categorical_cols if c in X_cols]

    # Initiate the model
    model = lgb.LGBMRegressor(
        objective=obj,
        n_estimators=800,
        random_state=SEED,
        n_jobs=-1,
        verbosity=-1
    )

    # Fit the model
    model.fit(X_train, y_train, categorical_feature=cat_in_X)

    # Predictions
    y_pred = np.round(model.predict(X_test), 3)

    # Impute missing values with the predictions
    df.loc[test_mask, col] = y_pred

df_imputed = df.copy()


In [451]:
df_raw['potassium_agri_use'].describe()

count    3.219000e+03
mean     2.226887e+05
std      9.484771e+05
min      0.000000e+00
25%      1.093595e+03
50%      1.179931e+04
75%      7.061900e+04
max      1.172799e+07
Name: potassium_agri_use, dtype: float64

In [452]:
df_imputed['potassium_agri_use'].describe()

count    3.267000e+03
mean     2.194173e+05
std      9.418626e+05
min      0.000000e+00
25%      9.488500e+02
50%      1.115400e+04
75%      6.724211e+04
max      1.172799e+07
Name: potassium_agri_use, dtype: float64

In [454]:
# Columns used to build the KNN distance space
feature_cols = [c for c in df_imputed.columns if c not in exclude_cols]

# Identify dtypes off your working df
num_cols = df_imputed.select_dtypes('number').columns.to_list()
num_cols = [c for c in num_cols if c not in exclude_cols and c not in categorical_cols]

cat_cols = [c for c in categorical_cols if c not in exclude_cols]

# encode categorical cols and scale numerical cols
preprocessor = ColumnTransformer(
    transformers = [
        ("cat", OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols),
        ("num", StandardScaler(), num_cols)
    ],
    remainder="drop",
    verbose_feature_names_out=False
)

# training set: all rows where the target is not missing
train_mask = df_imputed[knn_cols].notna().all(axis=1)
train_df = df_imputed.loc[train_mask, feature_cols].copy()

# test set
test_mask = df_imputed[knn_cols].isna().any(axis=1)
test_df = df_imputed.loc[test_mask, feature_cols]

# Fit the preprocessor
preprocessor.fit(train_df)

# Transform training set and test set
X_train = preprocessor.transform(train_df)
X_test = preprocessor.transform(test_df)

# Fit K-NN Imputer on training set and impute test set
imputer = KNNImputer(n_neighbors=5, weights="uniform")
imputer.fit(X_train)

X_test_imputed = imputer.transform(X_test)

# Get the names of features after preprocessing
feature_names = preprocessor.get_feature_names_out()

# Identify which positions correspond to our target columns
num_col_idx = [i for i,f in enumerate(feature_names) if f in num_cols] 

# Extract values corresponding to the target columns
X_test_imputed_num = X_test_imputed[:, num_col_idx]

# Inverse-scale the numeric columns back to original units
scaler = preprocessor.named_transformers_["num"]
X_test_imputed_num_original = scaler.inverse_transform(X_test_imputed_num)

# Convert into dataframe
num_imputed_df = pd.DataFrame(
    X_test_imputed_num_original,
    columns=num_cols,
    index=df_imputed.index[test_mask]
)

In [458]:
# List of columns to be imputed with KNN model
knn_cols = [
    'aoi_credit_to_ag_forest_fish', 'emission_share_land_use_change', 
    'total_fdi_inflows', 'total_pesticide_export_value', 'phosphorus_production',
    'emission_share_agri_waste_mgt', 'emission_share_farmgate', 
    'emission_share_energy_use', 'emission_share_crops', 
    'emission_share_pre_and_post_production', 'emission_share_end_to_end_agrifood',
    'emission_share_ipcc_agriculture', 'emission_share_livestock'
    ]

# In each of the missing column, impute NaNs with predictions
for col in knn_cols:
    target_col_idx = df_imputed.loc[df_imputed[col].isna()].index.to_list()
    df_imputed.loc[target_col_idx, col] = num_imputed_df.loc[target_col_idx, col].values

df_imputed.shape

(3267, 93)

In [457]:
# Check if all the missing values are imputed or not
df_imputed.isna().mean().sort_values(ascending=False).head(10)

area                                        0.0
potassium_agri_use                          0.0
cropland_phosphorus_use_efficiency          0.0
cropland_phosphorus_per_unit_area           0.0
cropland_nitrogen_use_efficiency            0.0
cropland_nitrogen_per_unit_area             0.0
potassium_use_per_value_of_ag_production    0.0
potassium_use_per_capita                    0.0
potassium_use_per_area_of_cropland          0.0
potassium_import_quantity                   0.0
dtype: float64

In [459]:
# Saving imputed dataset as csv file
df_imputed.to_csv('dataset_1_item_independent_imputed.csv', index=False)